In [40]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

sys.path.append('..')

from mim.cross_validation import ChronologicalSplit
import mim.extractors.ab_json as ab_json
import mim.models.ab_nn as ab_nn

In [23]:
specification = {
    "labels": {"target": "label-index+30d-ami+death-30d"},
    "index": {"json_train": "/home/sapfo/andersb/PycharmProjects/Expect/json_data/pontus_glukos/hbg+lund-train.json.gz"},
    "features":{
        "gender",
        "age",
        "bl-Glukos", "bl-TnT", "bl-Krea", "bl-Hb"
    },
}

params={
    #'model': ab_nn.ab_simple_lr,
    'epochs': 100,
    'batch_size': 32,
            # 'learning_rate': 0.01,
    'input_shape': {'numeric': (5,),
                    'categorical': (2,)},
    'compile_args': {'optimizer': 'sgd',
                     'loss': 'binary_crossentropy',
                     'metrics': ['accuracy', tf.keras.metrics.AUC()]}
}

test_size = 0.25


In [3]:
extractor = ab_json.ABJSONExtractor(specification)

In [4]:
q = extractor.get_data()

{'gender', 'bl-Hb', 'age', 'bl-Krea', 'bl-TnT', 'bl-Glukos'}
['age', 'bl-Glukos', 'bl-Hb', 'bl-Krea', 'bl-TnT']
['gender']


In [5]:
print(ab_json.NUMERICAL_FEATURES)

['age', 'bl-Glukos', 'bl-Krea', 'bl-TnT', 'bl-Hb']


In [10]:
q['x']['numeric']

array([[ 60.94726027,   9.8       , 131.        , 115.        ,
         26.        ],
       [ 86.20945205,   6.7       , 113.        ,  76.        ,
         23.        ],
       [ 57.30726027,   6.4       , 141.        ,  84.        ,
          7.        ],
       [ 62.74849315,   6.7       , 128.        ,  77.        ,
          7.        ],
       [ 47.76753425,   5.6       , 152.        ,  88.        ,
          4.        ]])

In [47]:
type(q['x'].shape)

dict

In [39]:
train, val = next(ChronologicalSplit(test_size).split(q))

In [13]:
keras_model = ab_nn.ab_simple_lr(q['x'].shape)

In [14]:
keras_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
categorical (InputLayer)        [(None, 2)]          0                                            
__________________________________________________________________________________________________
numeric (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 7)            0           categorical[0][0]                
                                                                 numeric[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 1)            8           concatenate[0][0]     

In [20]:
keras_model.compile(**params['compile_args'])

In [43]:
x = q['x'].as_dataset
y = q['y'].as_dataset

keras_model.fit(x=tf.data.Dataset.zip((x, y)).batch(params['batch_size']).prefetch(3), epochs=params['epochs'])

Epoch 1/100
149/149 [==============================] - 1s 7ms/step - loss: 7.2926 - accuracy: 0.8803
Epoch 2/100
149/149 [==============================] - 1s 7ms/step - loss: 6.4349 - accuracy: 0.8702
Epoch 3/100
149/149 [==============================] - 1s 7ms/step - loss: 6.5441 - accuracy: 0.8767
Epoch 4/100
149/149 [==============================] - 1s 8ms/step - loss: 6.9154 - accuracy: 0.8759
Epoch 5/100
149/149 [==============================] - 1s 8ms/step - loss: 6.9272 - accuracy: 0.8719
Epoch 6/100
149/149 [==============================] - 1s 8ms/step - loss: 6.3853 - accuracy: 0.8736
Epoch 7/100
149/149 [==============================] - 1s 8ms/step - loss: 5.7996 - accuracy: 0.8815
Epoch 8/100
149/149 [==============================] - 1s 8ms/step - loss: 7.0777 - accuracy: 0.8712
Epoch 9/100
149/149 [==============================] - 1s 7ms/step - loss: 6.1578 - accuracy: 0.8794
Epoch 10/100
149/149 [==============================] - 1s 8ms/step - loss: 6.6725 - accura

149/149 [==============================] - 2s 11ms/step - loss: 7.1562 - accuracy: 0.8754
Epoch 82/100
149/149 [==============================] - 2s 11ms/step - loss: 6.2960 - accuracy: 0.8738
Epoch 83/100
149/149 [==============================] - 1s 8ms/step - loss: 6.2249 - accuracy: 0.8731
Epoch 84/100
149/149 [==============================] - 1s 8ms/step - loss: 6.4744 - accuracy: 0.8723
Epoch 85/100
149/149 [==============================] - 2s 11ms/step - loss: 6.3495 - accuracy: 0.8796
Epoch 86/100
149/149 [==============================] - 2s 11ms/step - loss: 6.8627 - accuracy: 0.8752
Epoch 87/100
149/149 [==============================] - 2s 16ms/step - loss: 6.3427 - accuracy: 0.8712
Epoch 88/100
149/149 [==============================] - 1s 8ms/step - loss: 6.7104 - accuracy: 0.8767
Epoch 89/100
149/149 [==============================] - 1s 9ms/step - loss: 6.4833 - accuracy: 0.8750
Epoch 90/100
149/149 [==============================] - 1s 9ms/step - loss: 6.8883 - accur

In [50]:
q['x']['numeric']

AttributeError: 'TensorShape' object has no attribute 'items'